In [1]:
import numpy as np
import os
import time
import datetime
import threading
import pandas as pd
import pickle
from mediapipe.python.solutions import drawing_utils as mp_drawing
from mediapipe.python.solutions import pose as mp_pose
import matplotlib.pyplot as plt
import cv2
import mediapipe as mp
from typing import Mapping
mp_pose = mp.solutions.pose

from audSample import *
from createAudio import *


pygame 2.1.2 (SDL 2.0.18, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
audioFiles/15sec.wav exists
audioFiles/30sec.wav exists
audioFiles/45sec.wav exists
audioFiles/60sec.wav exists
audioFiles/75sec.wav exists
audioFiles/90sec.wav exists
audioFiles/105sec.wav exists
audioFiles/120sec.wav exists
audioFiles/135sec.wav exists
audioFiles/150sec.wav exists
audioFiles/165sec.wav exists
audioFiles/180sec.wav exists


In [2]:
!pip install scikit-learn==1.0.1

You should consider upgrading via the 'C:\Users\Prathmesh Waghmode\Envs\django-ml\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
classifier = pickle.load(open('save_model3.unknown', 'rb'))

In [4]:
# to calculate angle in three points
def getAngles(item, pt1, pt2, pt3):
    a = item[pt1]
    b = item[pt2]
    c = item[pt3]
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = (np.arccos(cosine_angle))*(180/np.pi)
    return angle

In [5]:
def chk_vis(lis):
    cnt = 0
    for obj in lis:
        if (obj.visibility>0.5): cnt+=1
    if (cnt==33): return True
    else: return False 

In [6]:
from timeit import default_timer as timer

PosesAudioLoc = "audioFiles/poses/"
SecAudioLoc = "audioFiles/seconds/"

In [7]:

recently = []
steady_time = []

def start_vid(frame_rate):
    secCheck = 0
    vid = cv2.VideoCapture(0)
    # frame_rate = 30
    prev = time.time()
    threadChk = 0
    while(True):
        
        # Capture the video frame
        # by frame
        time_elapsed = time.time() - prev
        c_time = datetime.datetime.now().strftime('%M')
        ret, frame = vid.read()
        # time_elapsed = time.time() - prev
        # ret, frame = self.video.read()
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            with mp_pose.Pose() as pose_tracker:
                result = pose_tracker.process(image=frame)
                # mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
                pose_landmarks = result.pose_landmarks
            output_frame = frame.copy()
            dict_obj = {}
            for i in range(0,33):
                
                dict_obj[i] = mp_drawing.DrawingSpec(color=mp_drawing.BLUE_COLOR)
            dict_obj[13] = mp_drawing.DrawingSpec(color=(0, 128, 0))
            dict_obj[15] = mp_drawing.DrawingSpec(color=(0, 128, 0))
            dict_obj[11] = mp_drawing.DrawingSpec(color=(0, 128, 0)) 
            x: Mapping[int, mp_drawing.DrawingSpec] = dict_obj
            # print(pose_landmarks.landmark)
            pose_name = ""
            if pose_landmarks is not None:
                mp_drawing.draw_landmarks(
                    image=output_frame,
                    landmark_list=pose_landmarks,
                    connections=mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=  x)
                assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
                if chk_vis(pose_landmarks.landmark): 
                    pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]
                    # output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
                    frame_height, frame_width = frame.shape[:2]
                    pose_landmarks *= np.array([frame_width, frame_height, frame_width])
                    # uncomment this when predicting
                    item = pose_landmarks
                    arr = np.array([getAngles(item, 13, 11, 12),
                    getAngles(item, 24, 12, 14),
                    getAngles(item, 15, 13, 11),
                    getAngles(item, 12, 14, 16),
                    getAngles(item, 11, 23, 25),
                    getAngles(item, 12, 24, 26),
                    getAngles(item, 23, 25, 27),
                    getAngles(item, 24, 26, 28)])
                    res = classifier.predict([arr]) # classified pose name
                    pose_name = res[0]
                # ####### #
            cv2.rectangle(output_frame, (0,0), (650, 50), (0,0,0), -1)
            output_frame = cv2.putText(output_frame, pose_name, (25,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            # cv2.imshow('frame', output_frame)

            c_time = timer()
            
            if pose_name != "":
                steady_time_now = timer()
                steadyF = 0
                if not steady_time:
                    steady_time.append(steady_time_now)
                elif (steady_time_now - steady_time[0])>3:
                    steadyF = 1
                else:
                    # print(steady_time_now-steady_time[0])
                    steady_time.append(steady_time_now)
                    
                output_frame = cv2.putText(output_frame, str("{:.0f}".format(steady_time_now-steady_time[0]))+" s", (300,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                if steadyF == 1:
                    countSay = "{:.0f}".format(steady_time_now-steady_time[0])
                    if not recently: # classify and say
                        t1 = threading.Thread(target=start_playlist, args=[[ f"{PosesAudioLoc}{pose_name}.wav"]])
                        t1.start()
                        threadChk = 1
                        recently.append(c_time)

                    elif secCheck==0:
                        if int(countSay)%15 == 0:
                            print(countSay)
                            # basePerform = "seconds complete"
                            # textList = [f"{countSay} {basePerform}"]
                            # createAudio(textList, f"{audiosLoc}{countSay}sec.wav")
                            t1 = threading.Thread(target=start_playlist, args=[[ f"{SecAudioLoc}{countSay}sec.wav"]])
                            t1.start()
                            recently.pop()
                            recently.append(c_time)
                            secCheck = 1
                        else:
                            secCheck = 0
                    
                    if secCheck==1 and int(countSay)%15 != 0: secCheck=0
                    
                    
            else:
                steady_time.clear()
                recently.clear()
            cv2.imshow('frame', output_frame)

                
                    
            
        
        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    if threadChk: t1.join()
    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()

In [8]:
import warnings
warnings.filterwarnings('ignore') 

start_vid(30)

Exception in thread Thread-5:
Traceback (most recent call last):
  File "c:\users\prathmesh waghmode\appdata\local\programs\python\python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\users\prathmesh waghmode\appdata\local\programs\python\python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\sem 7\mini-project\project\YogaPoseDetect\audSample.py", line 26, in start_playlist
    pygame.mixer.music.queue(playList[0])
IndexError: list index out of range
Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\users\prathmesh waghmode\appdata\local\programs\python\python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\users\prathmesh waghmode\appdata\local\programs\python\python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\sem 7\mini-project\project\YogaPoseDetect\audSample.py", line 26, in start_playlist
    pygame

15
